In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv(r"C:\Users\The Best\Desktop\Projects\Car Dheko\Merged car\cleaned_merged_car.csv")

# Clean numeric columns
numeric_cols = ['kilometers_driven', 'registration_year', 'price', 'seats', 'kms_driven', 'bhp', 'rpm']
for col in numeric_cols:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col].str.replace(',', '').str.replace('₹', '').str.strip(), errors='coerce')

# Handle categorical variables (if 'state' is present, convert it to category)
if 'state' in df.columns:
    df['state'] = df['state'].astype('category').cat.codes

# One-Hot Encoding for other categorical features
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'state' in categorical_cols:
    categorical_cols.remove('state')  # Exclude 'state' if it's already encoded

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Check for missing values in the DataFrame
print("Missing values in DataFrame:\n", df.isnull().sum())

# Option 1: Drop rows with NaN values in the target column
df = df.dropna(subset=['price'])

# Option 2: (Uncomment if you prefer to fill missing values)
# df['price'].fillna(df['price'].median(), inplace=True)  # Fill NaN with median value

# Split the data into features and target
X = df.drop('price', axis=1)
y = df['price']

# Check for any remaining NaN values in y
if y.isnull().any():
    print("NaN values found in target variable y.")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Selection and Training
model = RandomForestRegressor(random_state=42)

# Hyperparameter Tuning using Grid Search
param_grid = {
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:\n", grid_search.best_params_)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the modified DataFrame to a new CSV file
df.to_csv(r"C:\Users\The Best\Desktop\Projects\Car Dheko\Car_Modeldev.csv", index=False)

Missing values in DataFrame:
 kilometers_driven                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 666, in _validate_par

Best Parameters:
 {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error: 464346447.14349335


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import joblib

# Load the dataset
file_path = r'C:\Users\The Best\Desktop\Projects\Car Dheko\Merged car\merged_car.csv'
df = pd.read_csv(file_path)

# Display missing values
print("Missing values in DataFrame:")
print(df.isnull().sum())

# Function to convert prices with 'Lakh' and 'Crore' to numbers
def convert_price(price):
    price = price.replace('₹ ', '').replace(',', '').strip()  # Remove currency symbols and commas
    if 'Lakh' in price:
        return float(price.replace(' Lakh', '')) * 1e5  # Convert Lakh to numeric
    elif 'Crore' in price:
        return float(price.replace(' Crore', '')) * 1e7  # Convert Crore to numeric
    return float(price)  # Convert other plain numbers to float
# Check for columns with URLs or irrelevant data
print(df.head())  # Inspect the dataset to find the problematic column

# If you find a column with URLs (for example, 'car_link'), drop it
df = df.drop(columns=['car_link'], errors='ignore')  # Replace 'car_link' with the actual column name if needed

# Continue with the previous steps for feature and target preparation
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up Random Forest and GridSearchCV as before
rf = RandomForestRegressor()

# Hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'max_depth': [None, 10, 20, 30],
}

# Grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters from grid search:")
print(grid_search.best_params_)

# Make predictions
y_pred = grid_search.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

 


Missing values in DataFrame:
car_links                    0
fuel_type                    0
body_type                    4
kilometers_driven            0
transmission                 0
owner                        0
oem                          0
model                        0
year                         0
variant                      0
price                        0
registration_year           52
insurance_validity           4
fuel_type_overview           0
seats                        6
kms_driven                   2
rto                        894
comfort_convenience         68
interior_features           77
exterior_features           90
safety_features             96
entertainment_features    1095
mileage                   8369
engine                    8369
max_power                   60
torque                    8369
wheel_size                8369
bhp                       2319
rpm                       2319
consolidated_data            0
dtype: int64
                            

ValueError: 
All the 540 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\The Best\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_forest.py", line 363, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\The Best\AppData\Roaming\Python\Python3